In [1]:
import pandas as pd
import numpy as np 
from pathlib import Path
import os

In [2]:
# Create a file path to the two directories where the input file is stored
onedrive_base = Path(os.environ.get('onedrive' , 'C:/Users/azona/OneDrive - Cerity Partners'))
process_base = onedrive_base / 'Portfolio Management Team' / '2 - Monthly AUM and Performance' / 'DEV' / '2.1.1 - DEV Addepar Queries'
input_path = onedrive_base / process_base

In [3]:
os.chdir(input_path)
file_name = 'Core Summary.xlsx'
data = pd.read_excel(file_name)

In [4]:
# list of asset classes, these will be dropped and then summed again 
asset_class = ['Cash & Cash Equivalents', 'Fixed Income', 'Equities', 
              'Specialty', 'Private Markets']

In [5]:
data = data.drop(data[data['CP - Asset Classification'].isin(asset_class)].index)

In [6]:
data['benchmark_flag'] = np.where(
    data['Starting Adjusted Value (No Div, USD)'].isna(), 
    'Benchmark', 
    ' '
)

In [7]:
data['Benchmark Name'] = np.where(data['benchmark_flag'] == 'Benchmark', data['CP - Asset Classification'], ' ')

In [8]:
benchmark_dataset = data[data['benchmark_flag'] == 'Benchmark']

In [9]:
# Reset index to make it sequential
data = data.reset_index(drop=True)

# Fill benchmark name ONLY to the row directly above each benchmark row
for i in range(len(data)):
    # If current row is a benchmark
    if data.loc[i, 'benchmark_flag'] == 'Benchmark':
        # Get the benchmark name
        benchmark_name = data.loc[i, 'Benchmark Name']
        
        # Fill ONLY the row directly above (if it exists and is not Treasury Bills)
        if i > 0:
            if data.loc[i-1, 'CP - Asset Classification'] != 'Treasury Bills':
                data.loc[i-1, 'Benchmark Name'] = benchmark_name

In [10]:
data = data[data['benchmark_flag'] != 'Benchmark']
data = data.drop(['benchmark_flag'], axis=1)
#data_total_row = data[data['CP - Asset Classification'] == 'Total']
data = data[data['CP - Asset Classification'] != 'Total']

In [11]:
benchmark_data_cols_2_drop = ['CP - Asset Classification', 'Starting Adjusted Value (No Div, USD)',
       'Adjusted Net Cash Flow (YTD, No Div, USD)',
       'Adjusted Total Return (No Div, USD)', 'Adjusted Value (No Div, USD)',   
       '1 Yr. Adjusted Beta (USD, S&P 500 (Total Return), Monthly)',
       'Adjusted Beta (YTD, USD, S&P 500 (Total Return), Monthly)',
       '3 Yr. Adjusted Beta (USD, S&P 500 (Total Return), Monthly)',
       'benchmark_flag', '1 Yr. Adjusted TWR (No Div, USD)']

In [12]:
benchmark_dataset = benchmark_dataset.drop(columns = benchmark_data_cols_2_drop)

In [13]:
benchmark_dataset.columns = [
    col if col == 'Benchmark Name' else 'Benchmark - ' + col 
    for col in benchmark_dataset.columns
]

In [25]:
data = data.merge(right=benchmark_dataset , on='Benchmark Name' , how='left')

In [ ]:
data['YTD Difference'] = data['YTD'] - data['Benchmark - YTD']
data = data.drop('Adjusted Net Cash Flow (YTD, No Div, USD)', axis=1)
data['Net Cash Flow'] = data['Adjusted Value (No Div, USD)'] - data['Starting Adjusted Value (No Div, USD)'] - data['Adjusted Total Return (No Div, USD)']


In [ ]:
total_start_value = data['Starting Adjusted Value (No Div, USD)'].sum()
total_net_cash_flow_value = data['Net Cash Flow'].sum()
total_end_value = data['Adjusted Value (No Div, USD)'].sum()


In [42]:
data['AA %'] = data['Adjusted Value (No Div, USD)'] / total_end_value
data['Beginning + Flows Weighted'] = (data['Starting Adjusted Value (No Div, USD)'] + data['Net Cash Flow']) / (total_start_value + total_net_cash_flow_value)
data['Weighted Performance'] = data['YTD'] * data['Beginning + Flows Weighted']
data['Weighted Benchmark Performance'] = data['Benchmark - YTD'] * data['Beginning + Flows Weighted']
data['Weighted Over/Under Performance'] = data['Weighted Performance'] - data['Weighted Benchmark Performance']
data['Current Month - Difference'] = data['Adjusted TWR (Current Month, No Div, USD)'] - data['Benchmark - Adjusted TWR (Current Month, No Div, USD)']

In [44]:
columns_sequence = ['CP - Asset Classification', 'Benchmark Name', 
              'Starting Adjusted Value (No Div, USD)',
              'Net Cash Flow',
              'Adjusted Total Return (No Div, USD)', 
              'Adjusted Value (No Div, USD)',
              'AA %',
              'YTD', 
              'Benchmark - YTD', 
              'YTD Difference',
              'Beginning + Flows Weighted',
              'Weighted Performance',
              'Weighted Benchmark Performance',
              'Weighted Over/Under Performance',
              'Adjusted TWR (Current Month, No Div, USD)',
              'Benchmark - Adjusted TWR (Current Month, No Div, USD)',
              'Current Month - Difference',
              '1 Yr. Adjusted TWR (No Div, USD)',
              '3 Yr. Annualized Adjusted TWR (No Div, USD)',
              'Benchmark - 3 Yr. Annualized Adjusted TWR (No Div, USD)',
              ]
data = data[columns_sequence]

## Still need to group by asset class and get total rows for each asset class

In [ ]:
#data.to_excel('combined_output.xlsx', index=False)

In [ ]:
# Need to make "difference", "begining + flows weighted", "weighted performance", "weighted over/under performance"